# Introduction to Monte Carlo
## Sampling from discrete probability distributions
#### Martin Burke, August 2018

### Metropolis-Hastings algorithm

The Metropolis-Hastings algorithm is another method of obtaining samples from $F$. Like the basic rejection sampler it draws samples from a proposal distribution and accepts them with a given probability. However in this case the denominator of the acceptance probability equation is the previous sample $x_i$, which is also the basis for the next proposal. 
For example, in the simple rejection sampler we made proposals independently by drawing a random number between two and twelve. In this example we shall propose the new state $x_f$ by drawing a random number (which may be positive or negative) and adding it to $x_i$.

In [ ]:
# import some stuff for random number generation, analysis and plotting
import numpy as np
import pandas as pd
from plotnine import *
from plotnine.data import *

# likelihood function
def correct_likelihood(z):
    comb = 6 - abs(z - 7)
    return comb * 1/36


# likelihood estimator
def dodgy_likelihood(z):
    like = correct_likelihood(z)
    pert = (np.random.random() - 0.5) * 0.1
    return max(like + pert, 0)

In [ ]:
# metropolis hastings algorithm
PROPOSAL = 8    # jump size


def met_hastings_alg(steps, likelihood_function):
    #initialise
    xi = 2
    lik_xi = likelihood_function(xi)
    lik_err = lik_xi - correct_likelihood(xi)
    markov_chain = list()
    num_accept = 0
    # get some samples
    for x in range(1, steps + 1):
        # propose new state
        xf = xi + np.random.randint(-PROPOSAL, PROPOSAL+1)
        # evaluate likelihood
        lik_xf = likelihood_function(xf)
        # accept with mh probability
        accept = False
        mh_prop = lik_xf / lik_xi
        if mh_prop > 1:
            # accept automatically
            accept = True
        else:
            # accept sometimes
            if np.random.random() < mh_prop:
                accept = True
        # add sample to mc
        if accept:
            num_accept = num_accept + 1
        rowi = (x, xi, lik_xi, xf, lik_xf, accept, num_accept / x, lik_err)
        markov_chain.append(rowi)
        # update xi
        if accept:
            xi = xf
            lik_xi = lik_xf
            lik_err = lik_xi - correct_likelihood(xi)
    # create data frame and return
    return pd.DataFrame(markov_chain, columns=["i", "xi", "lik_xi", "xf", "lik_xf", "accept", "ar", "lik_err"])

In [ ]:
# run markov chain 
mc = met_hastings_alg(100000, dodgy_likelihood)
print(mc)
trace = ggplot(mc, aes(x = "i", y = "xi")) + geom_line()
p = ggplot(mc, aes(x = "xi", y = "..density..")) + geom_histogram(binwidth=1)
print(trace)